## 個人課題１
- Github上でGoogleが管理しているリポジトリの情報をスクレイピングして下記の情報を取得する
    - リポジトリ名
    - 主要な言語
    - スターの数
- 上記のデータを保存するためのDBを作成し，スクレイピングしたデータを保存する
    - GithubのAPIは使わないこと（スクレイピングの課題なので）
    - ちゃんと`time.sleep(1)`入れてね
- 保存したデータをSELECT文で表示する

In [1]:
import sqlite3
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
# 取得したリポジトリを保存するデータベースのテーブルを作成
path = ''
db_name = 'repositories.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL(RUBを操作するための言語)を実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()

    # SQL文の作成
    # テーブルの作成
    sql = 'CREATE TABLE repositories(id INTEGER PRIMARY KEY, name TEXT, language TEXT, stars REAL);'

    # SQL文の実行
    cur.execute(sql)

except sqlite3.Error as e:
    print('エラーが発生しました：', e)

finally:
    # DBへの接続を閉じる
    conn.close()

In [ ]:
path = ''
db_name = 'repositories.db'

total = 0

# aタグの中のaria-labelにstarsが含まれている場合にTrueを返す関数
def has_stars(tag):
    # a タグでなければスキップ
    if tag.name != "a":
        return False
    aria = tag.get("aria-label")
    return aria is not None and "stars" in aria

# 全てのリポジトリを取得するために全てのページにアクセスする
for page in range(1, 95):
    time.sleep(1)
    url = f"https://github.com/orgs/google/repositories?page={page}"
    response = requests.get(url)
    # ステータスコードが200以外の場合に例外を発生させる
    response.raise_for_status()
    print("ステータスコード:", response.status_code)
    print("-----")
    response.encoding = response.apparent_encoding 
    soup = BeautifulSoup(response.text, "html.parser")

    # リポジトリのブロックを取得するコードをかく
    # 各リポジトリが一つの<li>タグとしてまとまっているのでそれを取得
    repo_blocks = soup.find_all('li', class_="ListItem-module__listItem--k4eMk")
    for repo_block in repo_blocks:
        time.sleep(1)

        # リポジトリの名前を取得
        # aタグの中の指定したクラスの中にあるテキストを取得する
        a_tag = repo_block.find('a', class_="Title-module__anchor--GmXUE Title-module__inline--oM0P7")
        if a_tag:
            repo_name = a_tag.get_text()
        else:
            repo_name = None
        print(repo_name)

        # 主要な言語の取得
        # spanタグの中の指定したクラスの中にあるテキストを取得する
        language_tags = repo_block.find('span', class_='ReposListItem-module__Text_4--mkG7R')
        if language_tags:
            language = language_tags.get_text()
        else:
            language = None
        print(language)

        # スターの数の取得
        star_tags = repo_block.find(has_stars)
        if star_tags:
            stars_num = star_tags.get_text()  
        else:
            stars_num = None
        print(stars_num)

        # 取得したリポジトリの名前と主要な言語とスターの数をデータベースに保存
        try:
            # DB接続オブジェクトの作成
            conn = sqlite3.connect(path + db_name)

            # SQL(RUBを操作するための言語)を実行するためのカーソルp武ジェクトを取得
            cur = conn.cursor()

            # データ挿入のSQL文
            sql = "INSERT INTO repositories (name, language, stars) VALUES (?, ?, ?);"

            repositories = [
                (repo_name, language, stars_num)
            ]

            cur.executemany(sql, repositories)

            # 変更をDBに反映させる
            conn.commit()
            
        except sqlite3.Error as e:
            print('エラーが発生しました：', e)

        finally:
            # DBへの接続を閉じる
            conn.close()
        
        print("--------")
        total += 1
    # 途中経過の表示
    print("-----")
    print(f"【ページ {page} 終了】 現在までの取得件数：{total} 件")

print("==== 全ページ取得完了 ====")
print(f"総件数：{total} 件")

ステータスコード: 200
-----
clusterfuzz
Python
5.5k
--------
adk-java
Java
930
--------
conscrypt
Java
1.4k
--------
or-tools
C++
13k
--------
workflow-graph
TypeScript
149
--------
dagger
Java
18k
--------
perfetto
C++
5k
--------
nearby
C++
879
--------
crubit
C++
924
--------
osv-scalibr
Go
532
--------
heir
C++
605
--------
site-kit-wp
JavaScript
1.3k
--------
oss-fuzz
Shell
12k
--------
horologist
Kotlin
656
--------
dawn
C++
768
--------
xls
C++
1.4k
--------
adk-samples
Python
6.4k
--------
nomulus
Java
1.8k
--------
orbax
Python
452
--------
earthengine-catalog
Jsonnet
121
--------
open-dice
C++
26
--------
yggdrasil-decision-forests
C++
620
--------
device-infra
Java
58
--------
gemma.cpp
C++
6.6k
--------
ground-android
Kotlin
272
--------
koladata
C++
27
--------
j2cl
Java
1.3k
--------
garf
Python
9
--------
highway
C++
5.1k
--------
googletest-rust
Rust
391
--------
-----
【ページ 1 終了】 現在までの取得件数：30 件
ステータスコード: 200
-----
testrun
Python
42
--------
error-prone
Java
7.1k
--------
skia
C